In [8]:
import time
import csv
import re

import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

import pprint

import pycountry

from bs4 import BeautifulSoup
from googlesearch import search

from tqdm.auto import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_country(txt):
    for c in pycountry.countries:
        if c.name in txt:
            return c.name
    return ''
CLEANR = re.compile('<.*?>')

In [3]:
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '\n', str(raw_html))
  cleantext = ' '.join([str(x).strip() for x in cleantext.split('\n')])
  return cleantext

In [4]:
def name_convert(self):
    searchval = 'yahoo finance ticker '+self
    link = []
    for url in search(searchval, lang='en', num_results=5):
        link.append(url)
    links = [x if 'finance.yahoo.com' in x else '' for x in link]
    tickers = []
    for link in links:
        try:
            link = str(link)
            link = link.split("/")
            if link[-1]=='':
                ticker=link[-2]
            else:
                x=link[-1].split('=')
                ticker=x[-1]
            tickers.append(ticker)
        except:
            continue
    return list(set(tickers))

In [5]:
def yahoo_extract(tickers=None):
    if isinstance(tickers, str):
        tickers = [tickers]
    data = []
    for ticker in tickers:
        try:
            info_lst = []
            url = 'https://finance.yahoo.com/quote/{}/profile?p={}'.format(ticker, ticker)
            r = requests.get(url, headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
            soup = BeautifulSoup(r.text, 'html.parser')
            for info in soup.find_all('div', {'class': 'C($tertiaryColor) Fz(12px)'}):
                info_lst.append(info.text)
            container = soup.find('div', {'class': 'asset-profile-container'})
            for info in container.find_all('span', {'class': 'Fw(600)'}):
                info_lst.append(info.text)
            for info in soup.find_all('p', {'class': 'D(ib) W(47.727%) Pend(40px)'}):
                info = cleanhtml(info)
                info_lst.append(get_country(info))
            data.append(info_lst)
        except:
            continue
    return data

In [6]:
companies = ["Harley-Davidson Inc. (Ticker: HOG)",
"Amtek Auto Ltd",
"Renault SA (Ticker: RNO)",
"Continental AG (Ticker: CON)",
"The Kroger Co. (Ticker: KR)",
]

companies = {k:v for k,v in enumerate(companies)}

companies = pd.DataFrame(companies.items(), columns=['id','company'])

companies.head()

,id,company
0,0,Harley-Davidson Inc. (Ticker: HOG)
1,1,Amtek Auto Ltd
2,2,Renault SA (Ticker: RNO)
3,3,Continental AG (Ticker: CON)
4,4,The Kroger Co. (Ticker: KR)


In [9]:
companies['tickers'] = companies.company.progress_apply(name_convert)
companies['data'] = companies.tickers.progress_apply(yahoo_extract)

_df = pd.DataFrame(companies.data.tolist())
_df.columns = ['data_{}'.format(x) for x in range(_df.shape[1])]
_df2 = pd.concat([companies, _df], axis=1)
_df2.fillna('', inplace=True)


def pandas_list_explode(df, col, prefix=None, drop_original=True, collist=[]):
    if prefix is None:
        prefix = col+'_'
    df[col] = df[col].apply(lambda x: '///'.join(x))
    _df = df[col].str.split('///', expand=True).add_prefix(prefix)
    _df.columns = [prefix+x for x in collist]
    df = df.join(_df)
    if drop_original:
        df.drop(col, axis=1, inplace=True)
    return df

collist = ['exchange','sector','industry','employees','country']
for col in _df.columns:
    _df2 = pandas_list_explode(_df2, col=col, collist=collist)

_df2.fillna('', inplace=True)
_df2

100%|██████████| 5/5 [00:45<00:00,  9.11s/it]


,id,company,tickers,data,data_0_exchange,data_0_sector,data_0_industry,data_0_employees,data_0_country,data_1_exchange,data_1_sector,data_1_industry,data_1_employees,data_1_country
0,0,Harley-Davidson Inc. (Ticker: HOG),"[chart, news, HAR.MU, HOG, HAR.TI]","[[NYSE - NYSE Delayed Price. Currency in USD, ...",NYSE - NYSE Delayed Price. Currency in USD,Consumer Cyclical,Recreational Vehicles,"5,800",United States,TLO - TLO Delayed Price. Currency in USD,,,"5,800",United States
1,1,Amtek Auto Ltd,"[history, AMTEKAUTO6.BO, AMTEKAUTO4.BO, news]",[],,,,,,,,,,
2,2,Renault SA (Ticker: RNO),"[community, RNO.PA, history, RNO.MI, fin-srch]",[[Paris - Paris Delayed Price. Currency in EUR...,Paris - Paris Delayed Price. Currency in EUR,Consumer Cyclical,Auto Manufacturers,"160,000",France,Milan - Milan Delayed Price. Currency in EUR,Consumer Cyclical,Auto Manufacturers,"160,000",France
3,3,Continental AG (Ticker: CON),"[CON.MU, history, CON.DE, holders, CON.BE]",[[XETRA - XETRA Delayed Price. Currency in EUR...,XETRA - XETRA Delayed Price. Currency in EUR,Consumer Cyclical,Auto Parts,"197,442",Germany,,,,,
4,4,The Kroger Co. (Ticker: KR),"[chart, KR, 0JS2.L]","[[NYSE - NYSE Delayed Price. Currency in USD, ...",NYSE - NYSE Delayed Price. Currency in USD,Consumer Defensive,Grocery Stores,"500,000",United States,LSE - LSE Delayed Price. Currency in USD,,,"500,000",United States


In [10]:
%run ~/postrun.ipynb

Author: Sparsh A.

Last updated: 2022-12-01 17:06:43

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

requests : 2.28.1
bs4      : 4.11.1
sys      : 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
pandas   : 1.4.3
pycountry: 22.3.5
re       : 2.2.1
lxml     : 4.8.0
numpy    : 1.23.5
csv      : 1.0

